<a href="https://colab.research.google.com/github/201921027/graduate_project_git/blob/main/gp_original_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

kogpt fine-tuning classification

https://github.com/ukairia777/tensorflow-nlp-tutorial/blob/main/22.%20Fine-tuning%20GPT-2%20(Cls%2C%20Chatbot%2C%20NLI)/22-5.%20kogpt2_kornli_tpu.ipynb

summarize 코드 참조

https://www.nbshare.io/notebook/764386829/Amazon-Review-Summarization-Using-GPT-2-And-PyTorch/

kakaoBrain kogpt3

https://github.com/kakaobrain/kogpt 

skt-ai kogpt2

https://github.com/SKT-AI/KoGPT2

runtime: ram 확장, GPU 가속

* **데이터 불러오기**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd '/content/drive/MyDrive/Colab Notebooks/graduation_project'

/content/drive/MyDrive/Colab Notebooks/graduation_project


In [ ]:
import csv

with open('/content/drive/MyDrive/Colab Notebooks/graduation_project/data/train_data_1-3.CSV', 'r') as data:
  train_data= csv.reader(data, delimiter=',')

with open('/content/drive/MyDrive/Colab Notebooks/graduation_project/data/test_data.csv', 'r') as data:
  test_data= csv.reader(data, delimiter=',')

In [ ]:
import pandas as pd

train_df= pd.read_csv('/content/drive/MyDrive/Colab Notebooks/graduation_project/data/train_data_1-3.CSV')
test_df= pd.read_csv('/content/drive/MyDrive/Colab Notebooks/graduation_project/data/test_data.csv')

In [ ]:
train_df= train_df[['title', 'text', 'emotion', 'emotion & summarize']]
test_df= test_df[['title', 'text', 'emotion']]
# train_df.head()

In [ ]:
train_df['emotion'].value_counts()

기쁨         241
슬픔         221
unknown     59
두려움         39
분노          38
Name: emotion, dtype: int64

In [ ]:
test_df['emotion'].value_counts()

기쁨         15
슬픔          5
unknown     5
분노          4
두려움         2
Name: emotion, dtype: int64

In [ ]:
train_df['train_text']=  "["+ train_df['title'].map(str) + "] " + train_df['text']
train_df['train_summary'] = train_df['emotion'] + '| ' + train_df['emotion & summarize']
train_df= train_df[['train_text', 'train_summary']]

test_df['test_text']= "["+ test_df['title'].map(str) + "] " + test_df['text']
test_df= test_df[['test_text']]

# train_df.head()
# test_df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
train_df.to_csv('/content/drive/MyDrive/Colab Notebooks/graduation_project/data/original-model_train_data.csv')
test_df.to_csv('/content/drive/MyDrive/Colab Notebooks/graduation_project/data/original-model_test_data.csv')

In [ ]:
train_data= pd.read_csv('/content/drive/MyDrive/Colab Notebooks/graduation_project/data/original-model_train_data.csv')
train_data= train_data[['train_text', 'train_summary']]
# train_data.head()

In [ ]:
# test_data= pd.read_csv('/content/drive/MyDrive/Colab Notebooks/graduation_project/data/original-model_test_data.csv')
# test_data= test_data[['test_text']]
# test_data.head()

# 아래 test section에서 수행

In [ ]:
# 결측값 제거 (단, 중복은 제거하지 않음)
def drop_na(df):
  df= df.dropna()
  df= df.reset_index(drop=True)
  return df

In [ ]:
train_data= drop_na(train_data)
# test_data= drop_na(test_data)

In [ ]:
# gpt2 max length = 1024
# summarize : 200자로 제한
# 본문은 800자로 제한
# summarize는 앞 200자를 자르고, 본문은 앞 400자, 뒤 400자를 잘라서 합침
# 스터디 할 때 어디서인가 들었는데, 
# 모델 용량때문에 데이터를 잘라야할 땐 앞뒤를 잘라서 합치는 게 
# 요약에 가장 성능이 좋다고 들었던 것 같아서...

In [ ]:
max(train_data['train_text'].apply(len))

9302

In [ ]:
i=0
for text in train_data['train_text']:
  if len(text) > 800:
    train_data['train_text'][i] = train_data['train_text'][i][0:400] + train_data['train_text'][i][len(text)-400:len(text)]
  i += 1

In [ ]:
max(train_data['train_text'].apply(len))

800

In [ ]:
max(train_data['train_summary'].apply(len))

210

In [ ]:
i=0
for text in train_data['train_summary']:
  if len(text) > 200:
    train_data['train_summary'][i] = train_data['train_summary'][i][0:200]
  i += 1

In [ ]:
max(train_data['train_summary'].apply(len))

200

In [ ]:
train_data['total']= train_data['train_text'] + ' TL;DR ' + train_data['train_summary']
train_data.head()

,train_text,train_summary,total
0,[걱정] 오늘도 일기를 쓰기 위해 연필을 들었다. 오늘은 2월 5일. 벌써 202...,두려움| 시간이 많이 흘렀는데 이룬 것이 별로 없는 것 같아 불안하시군요. 미루는 ...,[걱정] 오늘도 일기를 쓰기 위해 연필을 들었다. 오늘은 2월 5일. 벌써 202...
1,[20141008] 오늘 아침도 여전히 졸렸다. 잠을 적게 자는 것도 아니고 피곤...,분노| 엄청 싫어하는 친구가 있어서 짜증이 나는군요. 그 친구가 당신의 친구랑 같이...,[20141008] 오늘 아침도 여전히 졸렸다. 잠을 적게 자는 것도 아니고 피곤...
2,[나의 요즘] 사람과 사람이 만났을 때 가장 많이 하는 말은 '요즘 뭐하고 지내?...,두려움| 누워서만 지내서 몸은 편한데 마음이 불편하군요. 무언가 해야할 것 같은 조...,[나의 요즘] 사람과 사람이 만났을 때 가장 많이 하는 말은 '요즘 뭐하고 지내?...
3,[편입일기.. ] 평생을 열심히 살아본 적이 없다고 스스로 생각했다. 성공한 인생...,슬픔| 미래를 위해 미대 편입학원에 다니기 시작했군요. 하지만 편입 준비를 하는 것...,[편입일기.. ] 평생을 열심히 살아본 적이 없다고 스스로 생각했다. 성공한 인생...
4,[백수의 하루] 전날 난 일찍자고 일찍 일어나서 일반인의 하루를 보내리라 결심을 ...,두려움| 취업이 잘 되지 않아 힘든 상황이군요. 직업이 없는 상태라 하루하루 똑같은...,[백수의 하루] 전날 난 일찍자고 일찍 일어나서 일반인의 하루를 보내리라 결심을 ...


In [ ]:
print('total max length: ' , max(train_data['total'].apply(len)))

total max length:  1007


In [ ]:
# !git clone https://github.com/kakaobrain/kogpt.git

In [ ]:
# !git clone https://github.com/SKT-AI/KoGPT2.git

In [ ]:
# ! git clone https://github.com/jiminAn/Capstone_2022.git

* **pre-trained model(kakaoBrain kogpt) 불러오기**

In [ ]:
!pip install -r '/content/drive/MyDrive/Colab Notebooks/graduation_project/kogpt/requirements.txt'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 831.4 MB 6.8 kB/s 
     |████████████████████████████████| 3.1 MB 58.2 MB/s 
     |████████████████████████████████| 3.3 MB 43.3 MB/s 
     |████████████████████████████████| 101 kB 13.0 MB/s 
     |████████████████████████████████| 880 kB 65.3 MB/s 
     |████████████████████████████████| 596 kB 62.6 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=06e27009b32aec3e3fda36da24d7dab1537074961ba48c123423327c224981f2
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Attempting uninstall: torch
    Found existing installation: torch 1.12.1+cu113
    Uninstalling torch-1

In [ ]:
import torch
import tensorflow
# from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
# kogpt3

# pre-trained tokenizer 불러오기
# tokenizer = AutoTokenizer.from_pretrained(
#   'kakaobrain/kogpt', revision='KoGPT6B-ryan1.5b-float16',  # or float32 version: revision=KoGPT6B-ryan1.5b
#   bos_token='[BOS]', eos_token='[EOS]', unk_token='[UNK]', pad_token='[PAD]', mask_token='[MASK]'
# )

# # pre-trained model 불러오기
# model = AutoModelForCausalLM.from_pretrained(
#   'kakaobrain/kogpt', revision='KoGPT6B-ryan1.5b-float16',  # or float32 version: revision=KoGPT6B-ryan1.5b
#   pad_token_id=tokenizer.eos_token_id,
#   torch_dtype='auto', low_cpu_mem_usage=True
# )#.to(device='cuda', non_blocking=True)
# _ = model.eval()

In [ ]:
# kogpt2
from transformers import PreTrainedTokenizerFast

tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
  bos_token='</s>', eos_token='</s>', unk_token='<unk>',
  pad_token='<pad>', mask_token='<mask>')


Downloading:   0%|          | 0.00/2.69M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/0.98k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [ ]:
import torch
from transformers import GPT2LMHeadModel

model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')

Downloading:   0%|          | 0.00/490M [00:00<?, ?B/s]

In [ ]:
# model load
# model= GPT2LMHeadModel.from_pretrained('./checkpoint/model.pt').to(device)

In [ ]:
# model test

# text = '[평범한 일상] 오늘은 아주 무난한 하루를 보냈다. 팝송도 듣고, 산책도 하고 등등. 매일이 오늘 같았으면 좋겠다!'
# input_ids = tokenizer.encode(text, return_tensors='pt')
# gen_ids = model.generate(input_ids,
#                            max_length=128,
#                            repetition_penalty=2.0,
#                            pad_token_id=tokenizer.pad_token_id,
#                            eos_token_id=tokenizer.eos_token_id,
#                            bos_token_id=tokenizer.bos_token_id,
#                            use_cache=True)
# generated = tokenizer.decode(gen_ids[0])
# print(generated)

* 참고한 summarize 코드

https://towardsdatascience.com/question-answering-with-a-fine-tuned-bert-bc4dafd45626

https://medium.com/analytics-vidhya/an-ai-that-does-your-homework-e5fa40c43d17

https://www.nbshare.io/notebook/764386829/Amazon-Review-Summarization-Using-GPT-2-And-PyTorch/

In [ ]:
import numpy as np

In [ ]:
text_data= train_data['train_text']
summary_data= train_data['train_summary']

In [ ]:
from torch.utils.data import Dataset, DataLoader

max_len= len(max(train_data['total']))
extra_length= len(tokenizer.encode(" TL;DR "))

class textDataset(Dataset):  
    def __init__(self, tokenizer, text, max_len):
        self.max_len = max_len
        self.tokenizer = tokenizer
        self.eos = self.tokenizer.eos_token
        self.eos_id = self.tokenizer.eos_token_id
        self.text= text
        self.result = []

        for text in self.text:
            # Encode the text using tokenizer.encode(). We add EOS at the end
            tokenized = self.tokenizer.encode(text + self.eos)
            
            # Padding/truncating the encoded sequence to max_len 
            padded = self.pad_truncate(tokenized)            

            # Creating a tensor and adding to the result
            self.result.append(torch.tensor(padded))

    def __len__(self):
        return len(self.result)


    def __getitem__(self, item):
        return self.result[item]

    def pad_truncate(self, name):
        name_length = len(name) - extra_length
        if name_length < self.max_len:
            difference = self.max_len - name_length
            result = name + [self.eos_id] * difference
        elif name_length > self.max_len:
            result = name[:self.max_len + 3]+[self.eos_id] 
        else:
            result = name
        return result

In [ ]:
dataset= textDataset(tokenizer, train_data['total'], max_len)

In [ ]:
dataloader= DataLoader(dataset, batch_size= 1, drop_last=True) 

In [ ]:
# model load
# model= GPT2LMHeadModel.from_pretrained('./checkpoint/old_model.pt').to(device)

In [ ]:
def train(model, optimizer, dl, epochs):    
    for epoch in range(epochs):
        for idx, batch in enumerate(dl):
             with torch.set_grad_enabled(True):
                optimizer.zero_grad()
                batch = batch.to(device)
                output = model(batch, labels=batch)
                loss = output[0]
                loss.backward()
                optimizer.step()
                if idx % 50 == 0:
                    print("epoch: %f, loss: %f, %d"%(int(epoch), loss, idx))
                    model.save_pretrained('./checkpoint/old_model.pt') # pytorch

In [ ]:
import torch.optim as optim

optimizer= optim.AdamW(model.parameters(), lr= 3e-4)
model= model.to(device)

In [ ]:
train(model=model, optimizer=optimizer, dl=dataloader, epochs=30)

epoch: 0.000000, loss: 9.267990, 0
epoch: 0.000000, loss: 2.371522, 50
epoch: 0.000000, loss: 2.380273, 100
epoch: 0.000000, loss: 2.304886, 150
epoch: 0.000000, loss: 2.041277, 200
epoch: 0.000000, loss: 1.868050, 250
epoch: 0.000000, loss: 2.482947, 300
epoch: 0.000000, loss: 1.389516, 350
epoch: 0.000000, loss: 2.428214, 400
epoch: 0.000000, loss: 2.305589, 450
epoch: 0.000000, loss: 2.582309, 500
epoch: 0.000000, loss: 2.285297, 550
epoch: 1.000000, loss: 1.114740, 0
epoch: 1.000000, loss: 1.984914, 50
epoch: 1.000000, loss: 1.986743, 100
epoch: 1.000000, loss: 1.739288, 150
epoch: 1.000000, loss: 1.688375, 200
epoch: 1.000000, loss: 1.559651, 250
epoch: 1.000000, loss: 2.015565, 300
epoch: 1.000000, loss: 1.091560, 350
epoch: 1.000000, loss: 2.033303, 400
epoch: 1.000000, loss: 1.900468, 450
epoch: 1.000000, loss: 2.130802, 500
epoch: 1.000000, loss: 1.966061, 550
epoch: 2.000000, loss: 1.013857, 0
epoch: 2.000000, loss: 1.605938, 50
epoch: 2.000000, loss: 1.586382, 100
epoch: 2.0

In [ ]:
# model save
model.save_pretrained('./checkpoint/old_model.pt') # pytorch

In [ ]:
# model load
# model= GPT2LMHeadModel.from_pretrained('./checkpoint/old_model.pt').to(device)

In [ ]:
# model overfitting 확인

# 서버에서 아마 csv나 json으로 받아올텐데, 받아올 때 특수문자 처리 필수 (특히 따옴표 '', "")
# '[제목] 본문' 형태로 전처리
text = '[평범한 일상] 오늘은 아주 무난한 하루를 보냈다. 팝송도 듣고, 산책도 하고 등등. 매일이 오늘 같았으면 좋겠다!'
if len(text) + len(" TL;DR ") > 800:
  text = text[0:400] + text[len(text)-400:len(text)]
text= text + " TL;DR "

input_ids = tokenizer.encode(text, return_tensors='pt').to(device)
gen_ids = model.generate(input_ids,
                           max_length=len(input_ids[0])+ 50, # 50 -> 약 120자
                           repetition_penalty=2.0,
                           pad_token_id=tokenizer.pad_token_id,
                           eos_token_id=tokenizer.eos_token_id,
                           bos_token_id=tokenizer.bos_token_id,
                           use_cache=True).to(device)
generated = tokenizer.decode(gen_ids[0])
print(generated)
# print(len(text), len(generated))
# 왜 감정은 안 나오지?

[평범한 일상] 오늘은 아주 무난한 하루를 보냈다. 팝송도 듣고, 산책도 하고 등등. 매일이 오늘 같았으면 좋겠다! TL;DR unknown| 요즘 세상이 많이 추워보이는데 나는 아직 어릴 때부터 성격링을 하는 것이군요. 다이어리에서 나온 선택, 학벌같은 일은 정말 당하지 않은 것 같아요~ 그래서 널까 아니면 내가 마음을 굳어버리는


In [ ]:
TLDR_index= generated.index('TL;DR')
output= generated[TLDR_index+6:]
print('return comment:' , output)
# 어떻게 문장이 완결될 때 끝내지? 

return comment: unknown| 요즘 세상이 많이 추워보이는데 나는 아직 어릴 때부터 성격링을 하는 것이군요. 다이어리에서 나온 선택, 학벌같은 일은 정말 당하지 않은 것 같아요~ 그래서 널까 아니면 내가 마음을 굳어버리는


* test

In [ ]:
# model load
# model= GPT2LMHeadModel.from_pretrained('./checkpoint/old_model.pt').to(device)

In [ ]:
test_data= pd.read_csv('/content/drive/MyDrive/Colab Notebooks/graduation_project/data/original-model_test_data.csv')
test_data= test_data[['test_text']]
test_data.head()

In [ ]:
text_list = test_data['test_text']
summary_list=list()
# print(text_list)

In [ ]:
for i, text in enumerate(text_list):
  if len(text) + len(" TL;DR ") > 800:
    text = text[0:400] + text[len(text)-400:len(text)]
  text= text + " TL;DR "

  input_ids = tokenizer.encode(text, return_tensors='pt').to(device)
  gen_ids = model.generate(input_ids,
                            max_length=len(input_ids[0])+ 50, # 50 -> 약 120자
                            repetition_penalty=2.0,
                            pad_token_id=tokenizer.pad_token_id,
                            eos_token_id=tokenizer.eos_token_id,
                            bos_token_id=tokenizer.bos_token_id,
                            use_cache=True).to(device)
  generated = tokenizer.decode(gen_ids[0])
  TLDR_index= generated.index('TL;DR')
  output= generated[TLDR_index+6:]
  summary_list.append(output)
  if i%10 == 0:
    print('test {} 개 진행중'.format(i))

In [ ]:
# summary_list

['unknown| 열정과 애정을 주고 상대를 배려하게 한다 정말 좋은것을 하는건지라 믿어 의심치시는군요. 곁에 있다니 정말로 애증을 받을거에스가 있다고 생각해놓은명될꺼에요!</s>',
 'unknown| 취진 전에서부터 취직 준비라는 게 정말 신기 것 같아요! 언젠가 당신이 당신의 동생이 카페를 다니면서 활동적인건 가장 다른 경험이었군요, 무조건 뽑아서 지내봐주는 사람처럼 소통할 만한 사람이 되고',
 'unknown| 취진 전에서부터 취직 후까지의 것만으로도 큰 일이었군요. 그래도 자신의 마지막 시간에 누구랑 같이 느껴지는 날이 올거에요!</s>',
 'unknown| 앞길을 걸으면서 발을 놀랐군요. 좋은시간을 보냈길 바래요!</s>',
 'unknown| 성인이 되면 잠자리에 드는 마음으로 다른 사람처럼 변하는 건 정말 힘든 일 아닌데, 모든날에 돈주고 의지랑 고생 준비한 공통제를 억지로 낳으면 좋겠어요.</s>',
 'unknown| 나이를 먹어보느라 다른 것 같아요. 그래서부터는 활동 동안 스스로 시간을 보내는 것이 가장 괴로워하시고 한편으로는 생각하고 계시는군요? 당신이 재미있게 살았으면 됐겠어요.\n보여 금성 말고 일을 하는 삶을',
 'unknown| 취업이어서랑 헤어지는 느낌이 정말 슬펐겠군요. 좋은 사람이길 바라요</s>',
 'unknown| 나이를 열심히 투자하고 게임을 좋아하는 사람이 되었군요.. 정말 부럽네요. 남은 커피는 언제쯤 돈을 얻어가고제가 서투름 당신이 당신의 마음이 전해주고 있겠어서, 상실감을 배 것이라고 생각해보는 모습을 보면',
 'unkies 여자,남자 불러줬거든요~ 좋은 사람이 돼줄거예...어서 더더욱 좋아 보여 주말에도 팬덤, 당신이 버텨주세요. 뭐든 그들에게 벌 받아온다는건 알지만..... 그래서 널',
 'unknown| 당신이 버텨내운 모습을 보여도, 어쩔 수 없는게 대단한 일이라고 말해주고 있었군요. 그리고 평소와 달리 엄청 다른 사람들은 서로를 도와주 보면서 자신의 좋은 관계를 지속하는 것이 중요한 빛이 됐

In [ ]:
test_data['summary'] = summary_list
test_data.head()

,test_text,summary
0,[2018년 3월 29일 목요일] 나를 사랑하는 연습 어떻게 하면 나를 사랑할 수...,unknown| 열정과 애정을 주고 상대를 배려하게 한다 정말 좋은것을 하는건지라 ...
1,[김동률 오래된콘서트를 기념하며] 김동률이 부르는 노래와 나의 인연은 2015년 ...,unknown| 취진 전에서부터 취직 준비라는 게 정말 신기 것 같아요! 언젠가 당...
2,[성인간호학 중환자실 실습후기] 2017 1월 2일 월요일 수간호사선선생님의 ot...,unknown| 취진 전에서부터 취직 후까지의 것만으로도 큰 일이었군요. 그래도 자...
3,[반포한강에서] 바람이 꽤 쌀쌀해 콧물을 훌쩍이는 날씨에 반포한강에 왔다. 고속버...,unknown| 앞길을 걸으면서 발을 놀랐군요. 좋은시간을 보냈길 바래요!</s>
4,[햄버거를 먹었다.] 말 그대로 햄버거를 먹었다. 아버지와 둘이서. 원래 아버지는...,unknown| 성인이 되면 잠자리에 드는 마음으로 다른 사람처럼 변하는 건 정말 ...


In [ ]:
test_data.to_csv('/content/drive/MyDrive/Colab Notebooks/graduation_project/data/test_data_oldmodel_after.csv')

* evaluation

https://github.com/lovit/KoBERTScore

In [ ]:
# !git clone https://github.com/lovit/KoBERTScore

In [ ]:
%cd '/content/drive/MyDrive/Colab Notebooks/graduation_project/KoBERTScore'

/content/drive/MyDrive/Colab Notebooks/graduation_project/KoBERTScore


In [ ]:
!pip install -r '/content/drive/MyDrive/Colab Notebooks/graduation_project/KoBERTScore/requirements.txt'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!python setup.py install

running install
running bdist_egg
running egg_info
writing KoBERTScore.egg-info/PKG-INFO
writing dependency_links to KoBERTScore.egg-info/dependency_links.txt
writing entry points to KoBERTScore.egg-info/entry_points.txt
writing requirements to KoBERTScore.egg-info/requires.txt
writing top-level names to KoBERTScore.egg-info/top_level.txt
reading manifest template 'MANIFEST.in'
writing manifest file 'KoBERTScore.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build/bdist.linux-x86_64/egg
creating build/bdist.linux-x86_64/egg/KoBERTScore
copying build/lib/KoBERTScore/__init__.py -> build/bdist.linux-x86_64/egg/KoBERTScore
copying build/lib/KoBERTScore/tasks.py -> build/bdist.linux-x86_64/egg/KoBERTScore
copying build/lib/KoBERTScore/about.py -> build/bdist.linux-x86_64/egg/KoBERTScore
copying build/lib/KoBERTScore/cli.py -> build/bdist.linux-x86_64/egg/KoBERTScore
copying build/lib/KoBERTScore/score.py -> build/

In [ ]:
# %cd '/content/drive/MyDrive/Colab Notebooks/graduation_project'

In [ ]:
import csv
import pandas as pd

# references 불러오기
with open('/content/drive/MyDrive/Colab Notebooks/graduation_project/data/references.csv', 'r') as data:
  train_data= csv.reader(data, delimiter=',')

references_df= pd.read_csv('/content/drive/MyDrive/Colab Notebooks/graduation_project/data/references.csv') 

references_df= references_df[['title', 'summary']]

In [ ]:
with open('/content/drive/MyDrive/Colab Notebooks/graduation_project/data/test_data_oldmodel_after.csv', 'r') as data:
  train_data= csv.reader(data, delimiter=',')

candidates_df= pd.read_csv('/content/drive/MyDrive/Colab Notebooks/graduation_project/data/test_data_oldmodel_after.csv') 

candidates_df= candidates_df[['test_text', 'summary']]

In [ ]:
references_df.head()

,title,summary
0,2018년 3월 29일 목요일,당신은 예쁜 사람이니까 자책할 필요 없다고 생각해요. 당신을 사랑해줄 사람과 연애를...
1,김동률 오래된콘서트를 기념하며,김동률의 공연을 보며 크게 감명을 받았군요. 인생에서 좋았던 순간들에 한 장면이 추...
2,성인간호학 중환자실 실습후기,중환자실 실습을 했군요. 기관절개 한 곳으로 가래가 넘쳐 나온 걸 연차가 높으신 선...
3,반포한강에서,뱐포 한강에 가서 남준이의 노래를 들었군요. 남준이를 굉장히 좋아하나봐요. 그리고 ...
4,햄버거를 먹었다.,오랜만에 햄버거를 먹었군요! 햄버거를 묘사한 글에서 햄버거에 대한 당신의 애정이 느...


In [ ]:
candidates_df.head()

,test_text,summary
0,[2018년 3월 29일 목요일] 나를 사랑하는 연습 어떻게 하면 나를 사랑할 수...,unknown| 열정과 애정을 주고 상대를 배려하게 한다 정말 좋은것을 하는건지라 ...
1,[김동률 오래된콘서트를 기념하며] 김동률이 부르는 노래와 나의 인연은 2015년 ...,unknown| 취진 전에서부터 취직 준비라는 게 정말 신기 것 같아요! 언젠가 당...
2,[성인간호학 중환자실 실습후기] 2017 1월 2일 월요일 수간호사선선생님의 ot...,unknown| 취진 전에서부터 취직 후까지의 것만으로도 큰 일이었군요. 그래도 자...
3,[반포한강에서] 바람이 꽤 쌀쌀해 콧물을 훌쩍이는 날씨에 반포한강에 왔다. 고속버...,unknown| 앞길을 걸으면서 발을 놀랐군요. 좋은시간을 보냈길 바래요!</s>
4,[햄버거를 먹었다.] 말 그대로 햄버거를 먹었다. 아버지와 둘이서. 원래 아버지는...,unknown| 성인이 되면 잠자리에 드는 마음으로 다른 사람처럼 변하는 건 정말 ...


In [ ]:
# bert score
from KoBERTScore import BERTScore

model_name = "beomi/kcbert-base"
bertscore = BERTScore(model_name, best_layer=4)

Downloading:   0%|          | 0.00/250k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/619 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/438M [00:00<?, ?B/s]

Load beomi/kcbert-base with 4 layers


In [ ]:
# candidates_df['summary']
# references_df['summary']

In [ ]:
# references : 사람이 만든 요약문
# candidates : 기계가 만든 요약문

# list로 들어가야함
references = references_df['summary']
candidates = candidates_df['summary']

bertscore(references, candidates, batch_size=8)

Calculating BERTScore:   0%|          | 0/4 [00:00<?, ?it/s]
Train IDF: 1it [00:00, 28.23it/s]
Calculating BERTScore: 100%|██████████| 4/4 [00:00<00:00, 16.05it/s]


[0.43512198328971863,
 0.4564851224422455,
 0.47720834612846375,
 0.5189951658248901,
 0.39213478565216064,
 0.43747153878211975,
 0.3775472640991211,
 0.4029836058616638,
 0.4536148011684418,
 0.37325915694236755,
 0.44751209020614624,
 0.5291141867637634,
 0.46595367789268494,
 0.3765501081943512,
 0.49407440423965454,
 0.4440452456474304,
 0.3358968496322632,
 0.37318283319473267,
 0.43323031067848206,
 0.37105754017829895,
 0.46097442507743835,
 0.37384891510009766,
 0.44744569063186646,
 0.44189706444740295,
 0.3522997498512268,
 0.5498180985450745,
 0.4616355299949646,
 0.4249539375305176,
 0.4559708833694458,
 0.4739435315132141,
 0.368872731924057]